In [112]:
import os
import sys
import pickle
from copy import deepcopy
from collections import defaultdict

import numpy as np
import pandas as pd

import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch
import helper_functions as helper

In [2]:
# start a h2o instance to fit GLM models
# pick settings so things run fast but don't use all system resources
h2o.init(nthreads = 3, max_mem_size = "12G")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)56-b12)
  Starting server from S:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\AARONB~1\AppData\Local\Temp\tmpj0okuy9f
  JVM stdout: C:\Users\AARONB~1\AppData\Local\Temp\tmpj0okuy9f\h2o_Aaron_Blackwell_started_from_python.out
  JVM stderr: C:\Users\AARONB~1\AppData\Local\Temp\tmpj0okuy9f\h2o_Aaron_Blackwell_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Australia/Sydney
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,16 days
H2O cluster name:,H2O_from_python_Aaron_Blackwell_qcaavi
H2O cluster total nodes:,1
H2O cluster free memory:,10.67 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,3
H2O cluster status:,"accepting new members, healthy"


In [48]:
def import_data(data):    
    return pd.read_csv("train_test_data/" + data + ".csv", index_col="uid").drop("Unnamed: 0", axis="columns")        
X_train = import_data("X_train")
X_test = import_data("X_test")
y_train = import_data("y_train")
y_test = import_data("y_test")

y_train = y_train[[e for e in y_train.columns if 'post' in e]]
y_test  =  y_test[[e for e in y_test.columns  if 'post' in e]]
train_cols = X_train.columns.tolist()

In [49]:
# remove rows where targets all all NA
train = pd.concat([y_train, X_train], axis = 1)
train = train.dropna(how = 'all', subset = ['panas_pos_imp_post', 'panas_neg_imp_post', 'panas_pos_imp_post'])
test  = pd.concat([y_test, X_test], axis = 1)
test = test.dropna(how = 'all', subset = ['panas_pos_imp_post', 'panas_neg_imp_post', 'panas_pos_imp_post'])

In [50]:
# h2o likes to convert mostly na values into categories. so
# we copy the pandas type mapping across
col_types = dict(train.dtypes)
replacements = {'float64': 'real',
                'int64': 'int'}
for e in col_types:
    col_types[e] = replacements[str(col_types[e])]
    
train_h2o = h2o.H2OFrame(train, column_types = col_types)
test_h2o  = h2o.H2OFrame(test, column_types = col_types)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [111]:
all_models = dict()
best_models = dict()
# restrict search to only imputated cases where imputation actually occurs
targets = ['flourishing_scale_raw_class_post', 
           'flourishing_scale_raw_post', 
           'panas_neg_raw_class_post', 
           'panas_neg_raw_post',
           'panas_pos_raw_class_post',
           'panas_pos_raw_post',
           'panas_pos_imp_class_post',
           'panas_pos_imp_post',
          ]
for target in targets:
    print(f"searching for best model for target {target}")
    if 'class' in target: 
        families = ['binomial']
        metric_name = 'logloss'
    else:
        #TODO: tweedie_variance_power and tweedie_link_power (for tweedie) to work
        families = ["gaussian", "tweedie", "gamma", "poisson", "negativebinomial"]
        metric_name = 'mse'
    output_models = defaultdict(pd.DataFrame)
    best_metic_value = np.Inf
    best_family = None
    best_model = None
    for features in ['all', 'wk_10', 'wk_9-10']:
        if features == 'all':
            x_cols = train_cols
        elif features == 'wk_10':
            x_cols = [e for e in train_cols if 'wk_10' in e]
        elif features == 'wk_9-10':
            x_cols = [e for e in train_cols if 'wk_9' in e or 'wk_10' in e]
        else:
            raise ValueError('feature set not encoded')
        for family in families:
            print(f"searching for best model in {family} family")
            hyper_parameters = {'alpha': list(np.arange(0,1.1,0.1))}

            # h2o grid search doesn't support searching tweedie distribution over the 
            # space of canonical link functions so we define a custom search to support
            # this
            if family == "tweedie":
                # define a simple space (noting that both Guassian, Poisson and Gamma)
                # are already covered in other cases
                tweedie_variance_powers = [1.1, 1.3, 1.5, 1.7, 1.9]
            else:
                tweedie_variance_powers = [0]
            if family == "negativebinomial":
                hyper_parameters['theta'] = [1e-10, 1e-8, 1e-4, 1e-2, 0.1, 0.5, 1]

            for vp in tweedie_variance_powers:
                h2o_glm = H2OGeneralizedLinearEstimator(family = family, nfolds = 5, seed = 20191106, 
                                                        # tweedie parameters are ignored if not tweedie distn.
                                                        tweedie_variance_power = vp,
                                                        tweedie_link_power = 1.0 - vp)
                gs = H2OGridSearch(h2o_glm, hyper_parameters)

                gs.train(y = target, x = x_cols, training_frame = train_h2o)
                glm_grid_models = gs.get_grid(sort_by = 'mse')

                num_models = len(list(glm_grid_models.get_grid()))
                
                model_results = {
                    'response': target,
                    'family': family,
                    'alpha': [glm_grid_models.get_hyperparams(e)[0] for e in range(num_models)],
                    'best_lambda': [e.actual_params['lambda'][0] for e in glm_grid_models],
                    'metric_name': metric_name,
                    'features': features
                }
                if 'class' in target:
                    model_results['metric_value'] = list(glm_grid_models.get_grid(sort_by="mse").logloss(xval=True).values())
                else:
                    model_results['metric_value'] = list(glm_grid_models.get_grid(sort_by="mse").mse(xval=True).values())
                
                if family == "tweedie":
                    model_results['tweedie_power'] = vp
                elif family == "negativebinomial":
                    model_results['theta'] = [glm_grid_models.get_hyperparams(e)[1] for e in range(num_models)]
                # keep track of all models
                output_models[family] = output_models[family].append(pd.DataFrame(model_results), ignore_index = True)

                family_best_model = glm_grid_models.models[0]

                if 'class' in target:
                    if family_best_model.logloss(xval=True) < best_metic_value:
                        print(f"!! Classification new best model is {family} with {features} features !!")
                        print(f"old value {best_metic_value}, new value {family_best_model.logloss(xval=True)}")
                        best_model = family_best_model
                        best_metic_value = family_best_model.logloss(xval=True)
                        best_family = family
                else:
                    if family_best_model.mse(xval=True) < best_metic_value:
                        print(f"!! Regression new best model is {family} with {features} features !!")
                        print(f"old value {best_metic_value}, new value {family_best_model.mse(xval=True)}")
                        best_model = family_best_model
                        best_metic_value = family_best_model.mse(xval=True)
                        best_family = family
    all_models[target] = deepcopy(output_models)
    h2o.save_model(model=best_model, path=f"./fitted_models/h2o_glm/{target}", force=True)
    best_models[target] = {'best_model': best_model,
                           'metric_value': metric_name,
                           'best_metic_value': best_metic_value,
                           'best_family': best_family,
                           'features': 'features'}

searching for best model for target flourishing_scale_raw_class_post
searching for best model in binomial family
glm Grid Build progress: |████████████████████████████████████████████████| 100%
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
!! Classification new best model is binomial with all features !!
old value inf, new value 2.5403543680483622
searching for best model in binomial family
glm Grid Build progress: |████████████████████████████████████████████████| 100%
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
!! Classification new

Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperpar

Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperpar

Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperpar

Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperpar

...

Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperpar

Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperparameters: [alpha, theta]
Hyperpar

Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
!! Regression new best model is tweedie with all features !!
old value 114.95462183265509, new value 107.65724421310208
glm Grid Build progress: |████████████████████████████████████████████████| 100%
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
!! Regression new best model is tweedie with all features !!
old value 107.65724421310208, new value 101.76025622496753
searching for best model in gamma family
glm Grid Build progress: |████████████████████████████████████████████████| 100%
Hyperpa

Hyperparameters: [alpha]
Hyperparameters: [alpha]
glm Grid Build progress: |████████████████████████████████████████████████| 100%
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
glm Grid Build progress: |████████████████████████████████████████████████| 100%
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
searching for best model in gamma family
glm Grid Build progress: |████████████████████████████████████████████████| 100%
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters:

Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
glm Grid Build progress: |████████████████████████████████████████████████| 100%
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
glm Grid Build progress: |████████████████████████████████████████████████| 100%
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
Hyperparameters: [alpha]
searching for best model in gamma family
glm Grid Build progress: |█████████████████████

In [113]:
with open('fitted_models/h2o_glm/best_models.pkl', 'wb') as out_file:
    pickle.dump(best_models, out_file, protocol=pickle.HIGHEST_PROTOCOL)

In [114]:
all_models_pd = dict()
for e in all_models:
    all_models_pd[e] = pd.concat(all_models[e], ignore_index = True)
all_models_pd = pd.concat(all_models_pd, ignore_index = True, sort=False)
all_models_pd.sort_values(by=['response', 'metric_value'], inplace = True)
all_models_pd.to_csv("./fitted_models/h2o_glm/glm_cv_results.csv")
all_models_pd                      

S:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,response,family,alpha,best_lambda,metric_name,features,metric_value,theta,tweedie_power
7,flourishing_scale_raw_class_post,binomial,0.1,0.219293,logloss,all,1.422760e+00,NaN,NaN
11,flourishing_scale_raw_class_post,binomial,0.1,0.199049,logloss,wk_10,1.620012e+00,NaN,NaN
5,flourishing_scale_raw_class_post,binomial,0.2,0.109647,logloss,all,1.644682e+00,NaN,NaN
4,flourishing_scale_raw_class_post,binomial,0.3,0.073098,logloss,all,1.803371e+00,NaN,NaN
22,flourishing_scale_raw_class_post,binomial,0.1,0.199049,logloss,wk_9-10,1.817205e+00,NaN,NaN
8,flourishing_scale_raw_class_post,binomial,0.4,0.054823,logloss,all,1.977777e+00,NaN,NaN
12,flourishing_scale_raw_class_post,binomial,0.2,0.099525,logloss,wk_10,2.079857e+00,NaN,NaN
9,flourishing_scale_raw_class_post,binomial,0.5,0.043859,logloss,all,2.184116e+00,NaN,NaN
10,flourishing_scale_raw_class_post,binomial,0.0,0.021929,logloss,all,2.208055e+00,NaN,NaN
23,flourishing_scale_raw_class_post,binomial,0.2,0.099525,logloss,wk_9-10,2.269714e+00,NaN,NaN


In [100]:
for model in best_models:
    print(f'--------------- {model} ---------------')
    print(best_models[model]['best_model'])

--------------- flourishing_scale_raw_class_post ---------------
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  Grid_GLM_Key_Frame__upload_8a2e2651a46e7835ce81ece2ed24da8.hex_model_python_1574509614939_7667_model_2


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.1, lambda = 0.199 )",47,24,5,Key_Frame__upload_8a2e2651a46e7835ce81ece2ed24da8.hex




ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.129071859811097
RMSE: 0.3592657231230068
LogLoss: 0.42639762952682
Null degrees of freedom: 27
Residual degrees of freedom: 3
Null deviance: 38.81624211135692
Residual deviance: 23.878267253501917
AIC: 73.87826725350192
AUC: 0.9464285714285714
pr_auc: 0.8866301830096587
Gini: 0.8928571428571428

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3928962529952592: 


,,0,1,Error,Rate
0,0,13.0,1.0,0.0714,(1.0/14.0)
1,1,2.0,12.0,0.1429,(2.0/14.0)
2,Total,15.0,13.0,0.1071,(3.0/28.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,negativebinomial,log,"Elastic Net (alpha = 0.1, lambda = 2.1392 )",47,24,6,Key_Frame__upload_8a2e2651a46e7835ce81ece2ed24da8.hex




ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 19.0274678553048
RMSE: 4.3620485847024675
MAE: 3.628664331513762
RMSLE: 0.15034469332434386
R^2: 0.5843110721969529
Mean Residual Deviance: 0.5161531176559319
Null degrees of freedom: 29
Residual degrees of freedom: 5
Null deviance: 37.24941823174868
Residual deviance: 15.484593529677957
AIC: 229.3286446699923

ModelMetricsRegressionGLM: glm
** Reported on cross-validation data. **

MSE: 95.23984635771176
RMSE: 9.759090447255408
MAE: 7.394507607723188
RMSLE: 0.5595555809679155
R^2: -1.0806840887935873
Mean Residual Deviance: 6.28221304851128
Null degrees of freedom: 29
Residual degrees of freedom: 12
Null deviance: 38.33052952739586
Residual deviance: 188.4663914553384
AIC: 388.31044259565266

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,7.459499,3.3760016,10.1946535,6.268614,11.371713,6.5337906,2.9287238
1,mean_residual_deviance,6.20264,10.484555,24.89986,1.6465404,2.6335325,1.5237678,0.3094989
2,mse,100.36338,80.41161,195.31998,60.38851,175.81834,58.04743,12.242615
3,null_deviance,7.6661057,7.4500613,10.535039,19.088634,2.056067,5.985532,0.66525793
4,r2,-3.1112971,4.8202276,-2.1730683,-0.09034814,-11.558454,-1.735219,6.028674E-4
5,residual_deviance,37.69328,62.8903,149.39917,21.405025,7.9005976,9.142607,0.6189978
6,rmse,9.224835,4.3683205,13.975693,7.7710047,13.259651,7.618886,3.4989448
7,rmsle,0.41666797,0.4097932,1.1382723,0.26291674,0.30384338,0.26370144,0.114606



Scoring History: 


,,timestamp,duration,iterations,negative_log_likelihood,objective
0,,2019-11-24 18:33:13,0.000 sec,0,100.546735,3.351558
1,,2019-11-24 18:33:13,0.002 sec,1,90.155599,3.124782
2,,2019-11-24 18:33:13,0.004 sec,2,90.085450,3.124151
3,,2019-11-24 18:33:13,0.004 sec,3,90.085115,3.124151
4,,2019-11-24 18:33:13,0.006 sec,4,89.672280,3.118774
5,,2019-11-24 18:33:13,0.008 sec,5,89.671895,3.118774
6,,2019-11-24 18:33:13,0.009 sec,6,89.664322,3.118771


In [101]:
# make predictions for all models
all_predictions = dict()
for model in best_models:
    all_predictions[model] = best_models[model]['best_model'].predict(test_h2o)

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [102]:
# large number of models are just the constant model
# that is using linear regression we can't beat a straight line 
# without further feature engineering
# this makes sense (since non-linear behaviour is expected)
for model in best_models:
    print(f'--------------- {model} ---------------')
    print(all_predictions[model])

--------------- flourishing_scale_raw_class_post ---------------


predict,p0,p1
0,0.674595,0.325405
1,0.46531,0.53469
0,0.802689,0.197311
0,0.699222,0.300778
1,0.334784,0.665216
0,0.674595,0.325405
0,0.665073,0.334927
1,0.285583,0.714417
0,0.670316,0.329684



--------------- flourishing_scale_raw_post ---------------


predict
43.1739
43.1739
43.1739
43.1739
43.1739
43.1739
43.1739
43.1739
43.1739



--------------- panas_neg_raw_class_post ---------------


...

predict
29.3103
29.3103
29.3103
29.3103
29.3103
29.3103
29.3103
29.3103
29.3103



--------------- panas_pos_imp_class_post ---------------


predict,p0,p1
1,0.532378,0.467622
1,0.44491,0.55509
0,0.761994,0.238006
0,0.608341,0.391659
1,0.323016,0.676984
1,0.532378,0.467622
0,0.598196,0.401804
1,0.35816,0.64184
0,0.650723,0.349277



--------------- panas_pos_imp_post ---------------


predict
28.1871
30.9118
21.8521
25.9013
32.6991
28.1871
27.9981
29.3958
29.075


In [103]:
for target in best_models:
    print(f'--------------- {target} ---------------')
    print(best_models[target]['best_model'].model_performance(test_data=test_h2o))

--------------- flourishing_scale_raw_class_post ---------------

ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.2217936809674068
RMSE: 0.4709497648023691
LogLoss: 0.6459930272047623
Null degrees of freedom: 8
Residual degrees of freedom: -16
Null deviance: 12.476649250079015
Residual deviance: 11.62787448968572
AIC: 61.62787448968572
AUC: 0.575
pr_auc: 0.4560019841269841
Gini: 0.1499999999999999

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6652164872428995: 


,,0,1,Error,Rate
0,0,5.0,0.0,0.0,(0.0/5.0)
1,1,2.0,2.0,0.5,(2.0/4.0)
2,Total,7.0,2.0,0.2222,(2.0/9.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.665216,0.666667,1.0
1,max f2,0.197311,0.800000,7.0
2,max f0point5,0.665216,0.833333,1.0
3,max accuracy,0.665216,0.777778,1.0
4,max precision,0.714417,1.000000,0.0
5,max recall,0.197311,1.000000,7.0
6,max specificity,0.714417,1.000000,0.0
7,max absolute_mcc,0.665216,0.597614,1.0
8,max min_per_class_accuracy,0.665216,0.500000,1.0
9,max mean_per_class_accuracy,0.665216,0.750000,1.0



Gains/Lift Table: Avg response rate: 44.44 %, avg score: 41.42 %


...


Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.676984,0.500000,0.0
1,max f2,0.238006,0.714286,7.0
2,max f0point5,0.676984,0.714286,0.0
3,max accuracy,0.676984,0.777778,0.0
4,max precision,0.676984,1.000000,0.0
5,max recall,0.238006,1.000000,7.0
6,max specificity,0.676984,1.000000,0.0
7,max absolute_mcc,0.676984,0.500000,0.0
8,max min_per_class_accuracy,0.676984,0.333333,0.0
9,max mean_per_class_accuracy,0.676984,0.666667,0.0



Gains/Lift Table: Avg response rate: 33.33 %, avg score: 46.55 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.111111,0.674172,3.0,3.000000,1.0,0.676984,1.000000,0.676984,0.333333,0.333333,200.0,200.000000
1,,2,0.111111,0.671361,0.0,3.000000,0.0,0.000000,1.000000,0.676984,0.000000,0.333333,-100.0,200.000000
2,,3,0.111111,0.668549,0.0,3.000000,0.0,0.000000,1.000000,0.676984,0.000000,0.333333,-100.0,200.000000
3,,4,0.111111,0.665738,0.0,3.000000,0.0,0.000000,1.000000,0.676984,0.000000,0.333333,-100.0,200.000000
4,,5,0.111111,0.662926,0.0,3.000000,0.0,0.000000,1.000000,0.676984,0.000000,0.333333,-100.0,200.000000
5,,6,0.111111,0.648869,0.0,3.000000,0.0,0.000000,1.000000,0.676984,0.000000,0.333333,-100.0,200.000000
6,,7,0.222222,0.624490,0.0,1.500000,0.0,0.641840,0.500000,0.659412,0.000000,0.333333,-100.0,50.000000
7,,8,0.222222,0.589790,0.0,1.500000,0.0,0.000000,0.500000,0.659412,0.000000,0.333333,-100.0,50.000000
8,,9,0.333333,0.520103,0.0,1.000000,0.0,0.555090,0.333333,0.624638,0.000000,0.333333,-100.0,0.000000
9,,10,0.555556,0.467622,0.0,0.600000,0.0,0.467622,0.200000,0.561832,0.000000,0.333333,-100.0,-40.000000




--------------- panas_pos_imp_post ---------------

ModelMetricsRegressionGLM: glm
** Reported on test data. **

MSE: 54.06843750497589
RMSE: 7.3531243362924235
MAE: 5.966738140079889
RMSLE: 0.28870168806884644
R^2: -0.9551533204924274
Mean Residual Deviance: 1.6843525750972654
Null degrees of freedom: 8
Residual degrees of freedom: -16
Null deviance: 11.135365425332303
Residual deviance: 15.159173175875388
AIC: 112.99508313154182



In [104]:
all_coefs = list()
for target in best_models:
    all_coefs.append(
        pd.DataFrame.from_dict(best_models[target]['best_model'].coef(),
                               orient='index', 
                               columns = [target])
    )
    
all_coefs = pd.concat(all_coefs, axis = 1)
all_coefs.to_csv("./fitted_models/h2o_glm/glm_coefs.csv")
all_coefs

S:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,flourishing_scale_raw_class_post,flourishing_scale_raw_post,panas_neg_raw_class_post,panas_neg_raw_post,panas_pos_raw_class_post,panas_pos_raw_post,panas_pos_imp_class_post,panas_pos_imp_post
Intercept,-1.602945,0.686243,-2.037220,0.399438,-1.606435,0.713342,-1.586781,3.035710
activity_running_ratio_wk_1,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN
activity_running_ratio_wk_10,19.966503,0.000000,0.000000,0.000000,9.456084,0.000000,7.327408,0.136610
activity_running_ratio_wk_2,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN
activity_running_ratio_wk_3,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN
activity_running_ratio_wk_4,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN
activity_running_ratio_wk_5,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN
activity_running_ratio_wk_6,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN
activity_running_ratio_wk_7,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN
activity_running_ratio_wk_8,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN


In [ ]:
# TO DO create summary metrics and stuff
# only need to compare hyper-parameters for raw vs imputated at a high level

In [ ]:
# get top 10 parameters per model
# auc curve
# pvo
# confusion matrix